<h1><center>Next Word Prediction using Deep Learning</h1>

In [ ]:
# Import relevant libraries
import re

In [ ]:
# File path
file_path = 'Data/pg5200.txt'

# Open the file for reading
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the file
    text = file.read()

    # Split the text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    # Display first 10 sentences
    for sentence in sentences[:10]:
        print(sentence)

The Project Gutenberg eBook of Metamorphosis
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever.
You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org.
If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

*** This is a COPYRIGHTED Project Gutenberg eBook.
Details Below.
***
***     Please follow the copyright guidelines in this file.
     ***


Title: Metamorphosis

Author: Franz Kafka

Translator: David Wyllie

Release date: August 17, 2005 [eBook #5200]
                Most recently updated: April 28, 2021

Language: English



*** START OF THE PROJECT GUTENBERG EBOOK METAMORPHOSIS ***




Metamorphosis

by Franz Kafka

Translated by David Wyllie




I


One morning, when Gregor Samsa woke from troub